In [1]:
import os
import subprocess
import requests

# 比对 fasta 文件与数据库
def run_diamond(fasta_file, diamond_db):
    diamond_out = fasta_file + ".daa"
    cmd = "diamond blastp --query %s --db %s --out %s --outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore" % (fasta_file, diamond_db, diamond_out)
    subprocess.call(cmd, shell=True)
    return diamond_out

# 将 KEGG Gene ID 映射到 KEGG Pathway ID
def get_pathway_by_gene(gene_id):
    url = "http://rest.kegg.jp/link/pathway/%s" % gene_id
    pathway_id = None
    try:
        response = requests.get(url)
        if response.status_code == 200:
            pathway_id = response.text.strip().split("\t")[1].split(":")[1]
    except:
        pass
    return pathway_id

# 获取每个代谢通路的详细信息
def get_pathway_info(pathway_id):
    url = "http://rest.kegg.jp/get/%s" % pathway_id
    pathway_info = None
    try:
        response = requests.get(url)
        if response.status_code == 200:
            pathway_info = response.text
    except:
        pass
    return pathway_info

# 主程序
def main(fasta_file, diamond_db):
    diamond_out = run_diamond(fasta_file, diamond_db)
    pathway_ids = set()
    with open(diamond_out, "r") as f:
        for line in f:
            items = line.strip().split("\t")
            gene_id = items[0]
            pathway_id = get_pathway_by_gene(gene_id)
            if pathway_id:
                pathway_ids.add(pathway_id)
    pathway_infos = {}
    for pathway_id in pathway_ids:
        pathway_info = get_pathway_info(pathway_id)
        pathway_infos[pathway_id] = pathway_info
    return pathway_infos

if __name__ == "__main__":
    fasta_file = "input.fasta"
    diamond_db = "kegg.dmnd"
    pathway_infos = main(fasta_file, diamond_db)
    for pathway_id, pathway_info in pathway_infos.items():
        print("Pathway ID: %s" % pathway_id)
        print(pathway_info)


FileNotFoundError: [Errno 2] No such file or directory: 'input.fasta.daa'